In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import plotly.express as px

from datetime import datetime
from scipy import stats

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [5]:
#Pick Season
season = "2020-21"


#Import Files
all_players = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_players.csv', parse_dates= ["news_added"])
all_teams = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_teams.csv')
all_events = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_events.csv', parse_dates= ["deadline_time"])
player_types = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_types.csv')
game_phases = pd.read_csv(r'./data/' + season + '/Cleaned_data/game_phases.csv')

player_season_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_season_history.csv')
player_gameweek_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_gameweek_history.csv')
#player_gameweek_history = pd.read_csv(r'./data/2019-20/player_past_history/player_gameweek_history_raw.csv')
player_future_fixture = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_future_fixture.csv', parse_dates = ["kickoff_time"])




In [6]:
player_future_fixture.head()

,fixture_id,fixture_code,home_team_id,team_h_score,away_team_id,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty,player_id,opponent_team_id,team_id
0,2,2128288,8,NaN,1,NaN,1.0,False,0,False,2020-09-12 14:00:00,Gameweek 1,False,2,1,8,1
1,9,2128296,1,NaN,19,NaN,2.0,False,0,False,2020-09-19 14:00:00,Gameweek 2,True,2,1,19,1
2,23,2128310,11,NaN,1,NaN,3.0,False,0,False,2020-09-26 14:00:00,Gameweek 3,False,5,1,11,1
3,29,2128316,1,NaN,15,NaN,4.0,False,0,False,2020-10-03 14:00:00,Gameweek 4,True,3,1,15,1
4,44,2128331,12,NaN,1,NaN,5.0,False,0,False,2020-10-17 14:00:00,Gameweek 5,False,5,1,12,1


In [7]:
def player_future(player_id):
    data = player_future_fixture[player_future_fixture.player_id == player_id]

    player = all_players[all_players.player_id == player_id]
    team = data.team_id.iloc[0]
    goals_scored = player.iloc[0].goals_scored
    player_name = player.iloc[0]['web_name']

    fig = px.line(
        data, 
        x = 'event_name', 
        y = 'difficulty', 
        title = player_name,
        hover_data=['is_home'])

    fig.show()

In [8]:
player_future(150)

In [9]:
    opponent_strength = player_future_fixture['difficulty']
    opponent_strength
    #opponent_strength = player_analysis.iloc[:,2]

0        2
1        2
2        5
3        3
4        5
        ..
18311    3
18312    2
18313    3
18314    2
18315    4
Name: difficulty, Length: 18316, dtype: int64

In [10]:
def get_SMA(player_id , count):
    data = player_future_fixture[player_future_fixture.player_id == player_id]
    opponent_strength = data['difficulty']
    # List to store moving average results
    SMA2 = list(range(0, len(opponent_strength) - count))

    # Compute moving average
    for ii in range(len(SMA2)):
        # Previous gw index
        index = range(ii, ii + count)
        # Get data for previous gw
        prev_gw2 = opponent_strength.iloc[index]

        # Sum previous gw
        summation2 = np.sum(prev_gw2)

        # Get average
        avg2 = summation2/count
        # Save results to list
        SMA2[ii] = avg2
         
    # Define column label
    label2 = f"Opponent strength SMA"

    
    # Get corresponding gw for moving_avg
    gameweeks2 = opponent_strength.index[count:]
    gameweek = data.event_name[count:]
    
    # Convert list into Pandas Series
    SMA2 = pd.Series(SMA2, name = label2, index = gameweek)
    
    
    return SMA2

In [11]:
result = get_SMA(150,5)
result

event_name
Gameweek 6     2.8
Gameweek 7     2.4
Gameweek 8     2.6
Gameweek 9     3.0
Gameweek 10    3.0
Gameweek 11    2.6
Gameweek 12    2.8
Gameweek 13    2.8
Gameweek 14    2.8
Gameweek 15    3.0
Gameweek 16    3.4
Gameweek 17    3.6
Gameweek 18    3.4
Gameweek 19    3.2
Gameweek 20    3.0
Gameweek 21    2.8
Gameweek 22    2.4
Gameweek 23    2.4
Gameweek 24    2.6
Gameweek 25    2.6
Gameweek 26    3.0
Gameweek 27    3.2
Gameweek 28    3.6
Gameweek 29    3.4
Gameweek 30    3.4
Gameweek 31    2.8
Gameweek 32    2.6
Gameweek 33    2.4
Gameweek 34    2.6
Gameweek 35    2.6
Gameweek 36    2.6
Gameweek 37    2.8
Gameweek 38    2.8
Name: Opponent strength SMA, dtype: float64

In [12]:
player_future(150)

## FANTASY PLAYER VALUE

In [16]:
top_players = all_players[['web_name' , 'now_cost', 'status' , 'player_type_id', 'total_points' , 'minutes' , 'PP90' , 'PPG' , 'PPMM' , 'VAPM']][all_players['minutes']>1000][all_players['status'] != "i"].sort_values(by='VAPM' , ascending = False).head(60)
top_players.head(20)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
302,Fernandes,105,a,3,117,1187,8.871104,8.4,0.800000,0.609524
104,Alonso,60,a,2,100,1429,6.298111,5.6,0.933333,0.600000
387,Lloris,55,a,1,98,1808,4.878319,4.7,0.854545,0.490909
259,Alexander-Arnold,75,a,2,210,3173,5.956508,5.5,0.733333,0.466667
303,Martial,90,a,4,200,2625,6.857143,6.2,0.688889,0.466667
96,Pope,55,a,1,170,3420,4.473684,4.5,0.818182,0.454545
272,De Bruyne,115,a,3,251,2790,8.096774,7.2,0.626087,0.452174
305,Henderson,55,a,1,160,3240,4.444444,4.4,0.800000,0.436364
464,Boly,55,a,2,97,1980,4.409091,4.4,0.800000,0.436364
463,Doherty,60,a,2,167,2836,5.299718,4.6,0.766667,0.433333


In [17]:
top_players[top_players['player_type_id'] == 1].head(3)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
387,Lloris,55,a,1,98,1808,4.878319,4.7,0.854545,0.490909
96,Pope,55,a,1,170,3420,4.473684,4.5,0.818182,0.454545
305,Henderson,55,a,1,160,3240,4.444444,4.4,0.800000,0.436364


In [18]:
top_players[top_players['player_type_id'] == 2].head(5)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
104,Alonso,60,a,2,100,1429,6.298111,5.6,0.933333,0.600000
259,Alexander-Arnold,75,a,2,210,3173,5.956508,5.5,0.733333,0.466667
464,Boly,55,a,2,97,1980,4.409091,4.4,0.800000,0.436364
463,Doherty,60,a,2,167,2836,5.299718,4.6,0.766667,0.433333
255,Robertson,70,a,2,181,3111,5.236258,5.0,0.714286,0.428571


In [19]:
top_players[top_players['player_type_id'] == 3].head(5)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
302,Fernandes,105,a,3,117,1187,8.871104,8.4,0.800000,0.609524
272,De Bruyne,115,a,3,251,2790,8.096774,7.2,0.626087,0.452174
322,Ritchie,50,a,3,74,1312,5.076220,4.1,0.820000,0.420000
254,Salah,120,a,3,233,2879,7.283779,6.9,0.575000,0.408333
394,Son,90,a,3,169,2478,6.138015,5.6,0.622222,0.400000


In [20]:
top_players[top_players['player_type_id'] == 4].head(3)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
303,Martial,90,a,4,200,2625,6.857143,6.2,0.688889,0.466667
443,Antonio,65,a,4,111,1767,5.653650,4.6,0.707692,0.400000
201,Vardy,100,a,4,210,3032,6.233509,6.0,0.600000,0.400000


In [21]:
price_gk = top_players['now_cost'][top_players['player_type_id'] == 1].head(1).sum()
price_def = top_players['now_cost'][top_players['player_type_id'] == 2].head(5).sum()
price_mid = top_players['now_cost'][top_players['player_type_id'] == 3].head(4).sum()
price_fwd = top_players['now_cost'][top_players['player_type_id'] == 4].head(1).sum()


team_cost = price_gk + price_def + price_mid + price_fwd
print("Price for the best team is " + str(np.divide(team_cost,10)))

Price for the best team is 85.5


In [81]:
def player_value():

    fig = px.scatter(
        top_players, 
        x = 'now_cost', 
        y = 'VAPM',
        size = 'minutes',
        text = 'web_name',
        hover_data=['PPG'])

    fig.show()

In [82]:
player_value()